In [9]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 훈련 데이터와 레이블 분리
X = train_data.drop("label", axis=1)
y = train_data["label"]

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 훈련 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 훈련
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 검증 데이터에 대한 예측
val_predictions = model.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {accuracy}")

# 테스트 데이터에 대한 예측
test_data = scaler.transform(test_data)
test_predictions = model.predict(test_data)

# 결과를 제출 형식에 맞게 저장
submission = pd.DataFrame({"ImageId": range(1, len(test_predictions) + 1), "Label": test_predictions})
submission.to_csv("submission.csv", index=False)


/home/dummy/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/dummy/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/dummy/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/dummy/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sparse

Validation Accuracy: 0.9628571428571429


# 1회차 결과
- Test accuracy (Validation accuracy) : 0.9628
- kaggle submission 결과, score 0.96267 달성
- 보통 MNIST 데이터의 알고리즘으로 알려진 CNN이 아닌 random forest로 해결함

In [3]:
# 2회차
# digit recognizer에 제출할 파이썬 코드가 필요. 단, 코드 실행 시 accuracy가 0.97 이상 가질 것

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

# 데이터 불러오기
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 훈련 데이터와 레이블 분리
X = train_data.drop("label", axis=1).values.astype("float32") / 255.0
y = to_categorical(train_data["label"].values, num_classes=10)

# 데이터 전처리: 이미지 형태로 변환
X = X.reshape(-1, 28, 28, 1)

# 훈련 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="softmax"))

# 모델 컴파일
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

# 테스트 데이터 예측
X_test = test_data.values.astype("float32") / 255.0
X_test = X_test.reshape(-1, 28, 28, 1)
predictions = model.predict(X_test)

# 결과 저장
results = np.argmax(predictions, axis=1)
submission = pd.DataFrame({"ImageId": range(1, len(results) + 1), "Label": results})
submission.to_csv("submission.csv", index=False)

2024-01-01 15:24:36.353377: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 15:24:36.353416: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 15:24:36.354392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 15:24:36.436464: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


2024-01-01 15:24:39.925837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 15:24:39.941744: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 15:24:39.941793: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 15:24:39.942792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 15:24:39.942841: I tensorflow/compile

525/525 [==============================] - 4s 4ms/step - loss: 0.2180 - accuracy: 0.9344 - val_loss: 0.0786 - val_accuracy: 0.9758
Epoch 2/10
525/525 [==============================] - 2s 4ms/step - loss: 0.0641 - accuracy: 0.9797 - val_loss: 0.0849 - val_accuracy: 0.9725
Epoch 3/10
525/525 [==============================] - 2s 5ms/step - loss: 0.0435 - accuracy: 0.9862 - val_loss: 0.0589 - val_accuracy: 0.9801
Epoch 4/10
525/525 [==============================] - 6s 11ms/step - loss: 0.0330 - accuracy: 0.9898 - val_loss: 0.0443 - val_accuracy: 0.9863
Epoch 5/10
525/525 [==============================] - 11s 22ms/step - loss: 0.0253 - accuracy: 0.9919 - val_loss: 0.0340 - val_accuracy: 0.9889
Epoch 6/10
525/525 [==============================] - 7s 13ms/step - loss: 0.0183 - accuracy: 0.9942 - val_loss: 0.0388 - val_accuracy: 0.9875
Epoch 7/10
525/525 [==============================] - 9s 17ms/step - loss: 0.0144 - accuracy: 0.9949 - val_loss: 0.0360 - val_accuracy: 0.9879
Epoch 8/10
5

In [8]:
# "위에서 생성한 모델의 정확도를 확인 할 수 있는 파이썬 코드가 필요" 프롬프트에 대해 gpt가 생성한 코드

# 모델 평가
eval_result = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {eval_result[1]*100:.2f}%")

263/263 [==============================] - 1s 5ms/step - loss: 0.0489 - accuracy: 0.9863
Validation Accuracy: 98.63%


# 2회차 결과
- 모델 정확도 (Test Accuracy) : 98.63 %
- kaggle score : 0.9864
- 모델 정확도 확인을 위한 코드가 생성되지 않아, 추가로 생성이 필요했음

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# 데이터 불러오기
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 훈련 데이터와 레이블 분리
X = train_data.drop("label", axis=1).values.astype("float32") / 255.0
y = keras.utils.to_categorical(train_data["label"].values, num_classes=10)

# 데이터 전처리: 이미지 형태로 변환
X = X.reshape(-1, 28, 28, 1)

# 훈련 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 구성
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# 모델 컴파일
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

# 모델 평가
eval_result = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {eval_result[1]*100:.2f}%")

# 테스트 데이터 예측
X_test = test_data.values.astype("float32") / 255.0
X_test = X_test.reshape(-1, 28, 28, 1)
predictions = model.predict(X_test)

# 결과 저장
results = np.argmax(predictions, axis=1)
submission = pd.DataFrame({"ImageId": range(1, len(results) + 1), "Label": results})
submission.to_csv("submission.csv", index=False)


Epoch 1/10
525/525 [==============================] - 3s 5ms/step - loss: 0.2180 - accuracy: 0.9350 - val_loss: 0.0756 - val_accuracy: 0.9774
Epoch 2/10
525/525 [==============================] - 2s 5ms/step - loss: 0.0610 - accuracy: 0.9810 - val_loss: 0.0490 - val_accuracy: 0.9856
Epoch 3/10
525/525 [==============================] - 2s 5ms/step - loss: 0.0414 - accuracy: 0.9872 - val_loss: 0.0465 - val_accuracy: 0.9845
Epoch 4/10
525/525 [==============================] - 2s 5ms/step - loss: 0.0316 - accuracy: 0.9902 - val_loss: 0.0624 - val_accuracy: 0.9804
Epoch 5/10
525/525 [==============================] - 2s 5ms/step - loss: 0.0232 - accuracy: 0.9922 - val_loss: 0.0366 - val_accuracy: 0.9886
Epoch 6/10
525/525 [==============================] - 3s 5ms/step - loss: 0.0171 - accuracy: 0.9943 - val_loss: 0.0361 - val_accuracy: 0.9892
Epoch 7/10
525/525 [==============================] - 3s 5ms/step - loss: 0.0156 - accuracy: 0.9944 - val_loss: 0.0405 - val_accuracy: 0.9881
Epoch 

# 3회차 결과
- Test accuracy : 0.9901
- kaggle score : 0.9879
- gpu 사용 확인함
- 이전 정확도 대비 1.37% 향상